In [1]:
import warnings
warnings.filterwarnings('ignore')
import keras
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense,Dropout

Using TensorFlow backend.


In [2]:
node = "r183c12s04"
dataset = pd.read_csv('my_code/norm_dataset/'+node+'_normalised.csv')
input_data = dataset.drop(labels=['timestamp','label'], axis=1).astype('float64').values
labels = dataset.loc[:,'label'].astype(int).values
input_dim = input_data.shape[1]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size=0.10, shuffle=True, random_state=21)

In [4]:
#autoencoder model
autoencoder = Sequential()
autoencoder.add(Dense(100, activation='relu', input_dim=input_dim, name='encoder_1'))
autoencoder.add(Dense(80, activation='relu', name="encoder_2"))
autoencoder.add(Dropout(0.5))
autoencoder.add(Dense(60, activation='relu', name="encoder_3"))
autoencoder.add(Dense(40, activation='relu', name="encoder_4"))
autoencoder.add(Dense(20, activation='relu', name='code'))
autoencoder.add(Dense(40, activation='relu', name="decoder_1"))
autoencoder.add(Dense(60, activation='relu', name="decoder_2"))
autoencoder.add(Dense(80, activation='relu', name="decoder_3"))
autoencoder.add(Dropout(0.5))
autoencoder.add(Dense(100, activation='relu', name="decoder_4"))
autoencoder.add(Dense(input_dim, activation='sigmoid', name='output'))
autoencoder.compile(optimizer='adam', loss="mean_squared_error")
print(autoencoder.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Dense)            (None, 100)               12500     
_________________________________________________________________
encoder_2 (Dense)            (None, 80)                8080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
encoder_3 (Dense)            (None, 60)                4860      
_________________________________________________________________
encoder_4 (Dense)            (None, 40)                2440      
_________________________________________________________________
code (Dense)         

In [5]:
history = autoencoder.fit(x=X_train, y=X_train, epochs=100, validation_split=0.10, batch_size=75,shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 3855 samples, validate on 429 samples
Epoch 1/100
3855/3855 [==============================] - 1s 142us/step - loss: 0.0793 - val_loss: 0.0161
Epoch 2/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0166 - val_loss: 0.0144
Epoch 3/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0146 - val_loss: 0.0139
Epoch 4/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0138 - val_loss: 0.0139
Epoch 5/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0134 - val_loss: 0.0133
Epoch 6/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0115 - val_loss: 0.0094
Epoch 7/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0093 - val_loss: 0.0087
Epoch 8/100
3855/3855 [==============================] - 0s 33us/step - loss: 0.0087 - val_loss: 0.0082
Epoch 9/100
3855/3855 [==============================] - 0s 34us/step - 

Epoch 76/100
3855/3855 [==============================] - 0s 54us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 77/100
3855/3855 [==============================] - 0s 34us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 78/100
3855/3855 [==============================] - 0s 37us/step - loss: 0.0035 - val_loss: 0.0038
Epoch 79/100
3855/3855 [==============================] - 0s 48us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 80/100
3855/3855 [==============================] - 0s 36us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 81/100
3855/3855 [==============================] - 0s 34us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 82/100
3855/3855 [==============================] - 0s 41us/step - loss: 0.0035 - val_loss: 0.0039
Epoch 83/100
3855/3855 [==============================] - 0s 44us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 84/100
3855/3855 [==============================] - 0s 41us/step - loss: 0.0030 - val_loss: 0.0033
Epoch 85/100
3855/3855 [==============================]

In [6]:
loss = autoencoder.evaluate(X_test, X_test)
print(loss)

476/476 [==============================] - 0s 27us/step
0.0027286826022470197


In [54]:
autoencoder.save('./models/my_models/autoencoder_model_'+node+'.h5')

In [7]:
#classifier model
encoder = Model(inputs=autoencoder.get_layer("encoder_1").input, outputs=autoencoder.get_layer("code").output)
classifier = Sequential()
classifier.add(encoder)
classifier.add(Dense(20, activation='relu', name="hidden"))
classifier.add(Dense(1,activation="sigmoid", name="output"))
classifier.layers[0].name="encoder"
classifier.layers[0].trainable=False
classifier.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Model)              (None, 20)                28700     
_________________________________________________________________
hidden (Dense)               (None, 20)                420       
_________________________________________________________________
output (Dense)               (None, 1)                 21        
Total params: 29,141
Trainable params: 441
Non-trainable params: 28,700
_________________________________________________________________
None


In [8]:
classifier.fit(x=X_train, y=y_train, epochs=110, validation_split=0.10, batch_size=75,shuffle=True)

Train on 3855 samples, validate on 429 samples
Epoch 1/110
3855/3855 [==============================] - 0s 79us/step - loss: 0.3766 - acc: 0.9157 - val_loss: 0.3032 - val_acc: 0.9580
Epoch 2/110
3855/3855 [==============================] - 0s 18us/step - loss: 0.2052 - acc: 0.9777 - val_loss: 0.1754 - val_acc: 0.9790
Epoch 3/110
3855/3855 [==============================] - 0s 18us/step - loss: 0.1240 - acc: 0.9888 - val_loss: 0.1086 - val_acc: 0.9930
Epoch 4/110
3855/3855 [==============================] - 0s 16us/step - loss: 0.0801 - acc: 0.9933 - val_loss: 0.0757 - val_acc: 0.9930
Epoch 5/110
3855/3855 [==============================] - 0s 16us/step - loss: 0.0618 - acc: 0.9909 - val_loss: 0.0516 - val_acc: 0.9930
Epoch 6/110
3855/3855 [==============================] - 0s 18us/step - loss: 0.0467 - acc: 0.9943 - val_loss: 0.0417 - val_acc: 0.9930
Epoch 7/110
3855/3855 [==============================] - 0s 17us/step - loss: 0.0379 - acc: 0.9951 - val_loss: 0.0314 - val_acc: 0.9953
E

In [9]:
loss, accuracy = classifier.evaluate(X_test, y_test)

476/476 [==============================] - 0s 21us/step


In [10]:
print("loss: {:.4e}".format(loss))
print("accuracy: {:2.4f}%".format(accuracy*100))

loss: 6.1864e-04
accuracy: 100.0000%


In [41]:
classifier.save('./models/my_models/classifier_model_'+node+'.h5')

In [11]:
def assign_class(predicted_probabilities, threshold=0.5):
    classes = []
    for i in range(len(predicted_probabilities)):
        if(predicted_probabilities[i] >= threshold):
            classes.append(1)
        else:
            classes.append(0)
    return np.asarray(classes)

In [12]:
predict_prob = classifier.predict(X_test)

In [13]:
classes = assign_class(predict_prob)

In [14]:
from sklearn.metrics import accuracy_score
print("Accuracy :"+str(accuracy_score(y_test,classes)*100))
from sklearn.metrics import f1_score
print("f1Score: "+str(f1_score(y_test,classes,average='weighted')))


Accuracy :100.0
f1Score: 1.0


In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes)

array([[444,   0],
       [  0,  32]])

<h2> Test on Node r183c09s01 </h2>

In [16]:
df = pd.read_csv("./my_code/norm_dataset/r183c09s01_normalised.csv")

In [17]:
#drop timestamp col and divide df into anomaly df and no anomaly df
df = df.drop("timestamp",axis=1)
df_noAn = df[df.label==0]
df_An = df[df.label==1]
labels_noAn = df_noAn["label"]
labels_An = df_An["label"]

In [18]:
#Drop label col for prediction
df_noAn = df_noAn.drop("label",axis=1)
df_An = df_An.drop("label",axis=1)

In [22]:
predict_prob_noAn = classifier.predict(df_noAn)
predict_prob_An = classifier.predict(df_An)

array([[1.3411045e-06],
       [2.9802322e-08],
       [2.0861626e-07],
       ...,
       [6.5522472e-05],
       [0.0000000e+00],
       [8.4171070e-06]], dtype=float32)

In [20]:
classes_An = assign_class(predict_prob_An)
classes_NoAn = assign_class(predict_prob_noAn)

In [21]:
#metrics for df anomaly
from sklearn.metrics import accuracy_score
print("Accuracy :"+str(accuracy_score(labels_An,classes_An)*100))
from sklearn.metrics import f1_score
print("f1Score: "+str(f1_score(labels_An,classes_An)))


Accuracy :0.0
f1Score: 0.0


In [140]:
#metrics for total dataset
labels = df["label"]
df_total = df.drop("label",axis=1)

In [141]:
predict_prob = classifier.predict(df_total)
classes = assign_class(predict_prob)

In [142]:
from sklearn.metrics import confusion_matrix
confusion_matrix(labels, classes)

array([[3683,    0],
       [  75,    0]])

<h2> Test on Node r183c09s03 </h2>

In [118]:
df = pd.read_csv("./my_code/norm_dataset/r183c09s03_normalised.csv")

In [119]:
df = df.drop("timestamp",axis=1)
#metrics for total dataset
labels = df["label"]
df_total = df.drop("label",axis=1)

In [120]:
predict_prob = classifier.predict(df_total)
classes = assign_class(predict_prob)

In [121]:
from sklearn.metrics import confusion_matrix
confusion_matrix(labels, classes)

array([[3683,    0],
       [  75,    0]])